In [ ]:
import pandas as pd
animes = ["""
Naruto Uzumaki is a young ninja from the Hidden Leaf Village who begins his journey as an outcast, 
shunned by his community because he harbors the powerful Nine-Tailed Fox spirit sealed within him. Despite 
facing initial rejection, Naruto dreams of becoming the Hokage (village leader) and gaining the respect of his peers. 
Through intense training, forming deep friendships with teammates Sasuke and Sakura, and overcoming numerous challenging 
trials, Naruto gradually transforms from a mischievous underdog into a powerful and compassionate ninja. His story follows 
his growth from a troubled child to a heroic ninja who fights to protect his village and ultimately brings peace to the ninja 
world, demonstrating the power of perseverance, forgiveness, and understanding.
""",
"""
Monkey D. Luffy is a young pirate with an extraordinary dream: to find the legendary treasure known as One Piece and become the Pirate King. After eating a mystical Devil Fruit that grants him the ability to stretch his body like rubber, Luffy sets out to sea with an increasingly diverse and powerful crew. Their journey across the vast and dangerous Grand Line is filled with epic battles, incredible adventures, and a quest to challenge the World Government and the most powerful pirates. Each crew member has their own complex backstory and personal ambition, from the skilled swordsman Zoro to the brilliant navigator Nami. As they sail, they confront corrupt marines, rival pirate crews, and increasingly powerful enemies, all while challenging the established world order and fighting for freedom, friendship, and their individual dreams.
""",
"""
Gon Freecss is a young boy who discovers that his father, whom he believed was dead, is actually a legendary Hunter - a elite professional with extraordinary privileges who can access restricted areas and pursue rare treasures and information. Determined to follow in his father's footsteps, Gon takes the incredibly difficult Hunter Examination, where he meets and befriends other exceptional characters like Killua, a former assassin from a notorious family, Kurapika, who seeks revenge against a criminal group that destroyed his clan, and Leorio, an aspiring medical professional. The series explores complex themes of friendship, personal growth, and moral ambiguity as Gon and his friends navigate increasingly dangerous and morally complex challenges. From the Hunter Examination to the Chimera Ant arc, the story progressively becomes darker and more sophisticated, examining the depths of human nature, power, and psychological complexity.
"""]

animes = pd.DataFrame(animes)

In [ ]:
display(animes[0])

In [1]:
import os
import pandas as pd
from groq import Groq
from pydantic import BaseModel, ValidationError
from typing import Literal
import time

new_model = "llama-3.3-70b-versatile"
old_model = "llama-3.2-3b-preview"

class Synopsis(BaseModel):
    summary: str = 'error summarizing'
    
df = pd.read_csv('C:/Users/Harvey/Downloads/anime_summaries_2.csv')

client = Groq(
    api_key="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",
)

#for synopsis in df['synopsis']

def sentiment_analysis(text):
    
    MAX_RETRIES = 3
    retry_count = 0
    while retry_count < MAX_RETRIES:
        
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                    
                    Please summarize the following anime synopsis into 1-2 concise sentences that capture the essence of the story. 
                    The summary should emphasize key themes and emotions to facilitate sentiment analysis. Return the summary in the 
                    following JSON format: {'summary: short summary here'}. STRICTLY RETURN ACCORDING TO JSON FORMAT

                    The anime synopsis: {text}
                    """,
                }
            ],
            model=new_model,
        )
    
        print(chat_completion.choices[0].message.content)
    
        try:
                # Attempt to validate the JSON
                summary = Synopsis.model_validate_json(chat_completion.choices[0].message.content)
                return summary
                break  # Success, exit the loop
        except ValidationError as e:
                retry_count += 1
                if retry_count < MAX_RETRIES:
                    print(f"Retrying... ({retry_count}/{MAX_RETRIES})")
                    time.sleep(1)  # Wait for 2 seconds before retrying
                else:
                    print("Validation Error:", e)
                    print("Maximum retries reached. Unable to get a valid response.")

df['summary'] = df['synopsis'].apply(sentiment_analysis)
#animes['summary'] = animes[0].apply(sentiment_analysis)

{"summary": "The anime is the third season of Huyao Xiao Hongniang, continuing the story with its key themes and emotions, expected to evoke a range of sentiments from viewers. It likely explores fantasy, adventure, and romance, given the context of the series."}
{"summary": "The anime revolves around the pursuit of the 'Eye of Rainbow' treasure, which has the power to make every dream come true, sparking a battle between Arale and her friends, and Bisna, who seeks to revive the devil and the Nanaba Kingdom, unleashing themes of friendship, ambition, and the blurred lines between good and evil."}
{"summary": "Lupin III's treasure hunt is disrupted by a time-traveling seeker of revenge, Mamo Kyosuke, who sends him 500 years into the past, where he must survive a treacherous historical era to secure his future, amidst a backdrop of mystery, adventure, and chaos."}
{"summary": "The anime celebrates the 25th anniversary of Chibi Maruko-chan with a planetarium show, evoking nostalgia and jo